In [1]:
# !pip install safetensors
# !pip install omegaconf
!pip install pytorch_lightning

In [8]:
import torch
import numpy as np

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

pipeline = StableDiffusionPipeline.from_single_file(
    "./models/checkpoint/dreamshaper_8.safetensors", use_safetensors=True
)
# pipeline.load_lora_weights("./models/lora/shinkai_makoto_offset.safetensors")

# lora_model_path = "./models/lora/shinkai_makoto_offset.safetensors"
# pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
# pipeline.unet.load_attn_procs(lora_model_path)
pipe = pipeline.to("cuda")

pipeline.config


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/junlei/anaconda3/envs/diffusion_env/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


FrozenDict([('vae', ('diffusers', 'AutoencoderKL')),
            ('text_encoder', ('transformers', 'CLIPTextModel')),
            ('tokenizer', ('transformers', 'CLIPTokenizer')),
            ('unet', ('diffusers', 'UNet2DConditionModel')),
            ('scheduler', ('diffusers', 'PNDMScheduler')),
            ('safety_checker',
             ('stable_diffusion', 'StableDiffusionSafetyChecker')),
            ('feature_extractor', ('transformers', 'CLIPFeatureExtractor')),
            ('requires_safety_checker', True)])

In [3]:
prompt = "overall view,,beach background,outdoors,no humans, dog, book, animal focus, open book, shiba inu, animal, wooden floor, photo background, realistic, reading "
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, bad feet, "
height = 512

width = 512

generate_picture_nums = 1

num_inference_steps = 100

guidance_scale = 7.5

generator = torch.manual_seed(32)

batch_size = 1

In [ ]:

images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=num_inference_steps,
    num_images_per_prompt=1,
    generator=torch.manual_seed(32)
).images


img_array = np.array(images[0])
img_array_2d = img_array.reshape(-1, img_array.shape[-1])
np.savetxt("image_rgb_info2.txt", img_array_2d, fmt="%d")

images[0]

!!!!!!!!!!!! cuda:0


  0%|          | 0/100 [00:00<?, ?it/s]

sampel tensor([[[[-1.8047, -0.3877,  0.9642,  ..., -0.1266, -0.6061,  0.2423],
          [ 0.0385,  0.1163, -0.6487,  ...,  0.9071, -1.1645,  0.2588],
          [ 0.1654, -2.4366,  1.4403,  ...,  1.1008, -0.2667, -0.5148],
          ...,
          [ 0.0326, -1.8886, -0.7849,  ...,  0.4783, -0.1841,  1.3131],
          [ 0.5335, -0.1040, -2.1922,  ..., -0.3814, -0.3455,  1.3013],
          [ 0.7521, -0.3136, -0.3970,  ..., -0.4423, -0.5648,  0.2351]],

         [[-0.2412, -0.0554, -0.3119,  ...,  1.8868,  0.7475, -0.4076],
          [-1.3582,  0.1406,  1.7211,  ...,  0.9861,  0.0198,  0.4988],
          [ 0.2824,  1.3598,  1.1226,  ...,  0.2526,  0.0765,  0.9735],
          ...,
          [-0.3796,  0.2718, -0.4460,  ..., -1.8158,  0.5586, -0.4551],
          [-1.1392,  1.1071,  0.7739,  ..., -1.7677,  0.2487,  1.4156],
          [ 0.6802,  1.6569, -0.7364,  ..., -0.7937, -0.9303, -0.3231]],

         [[-1.0222,  1.0846,  1.1355,  ..., -0.3750,  0.3913, -0.1355],
          [-0.4032, -0.

In [ ]:
import torch
import numpy as np
from PIL import Image
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 1. Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_single_file("runwayml/stable-diffusion-v1-5", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet")

scheduler = LMSDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

In [ ]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

生成图片的相关参数

In [ ]:
'''多张图片grid'''
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
'''采样过程可视化'''

def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

In [ ]:
'''get the text_embeddings for the prompt'''
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

In [ ]:
# generate the intial random noise
latents = torch.randn(
    (batch_size, unet.in_channels, height // 8, width // 8),
    generator=generator,
)
latents = latents.to(torch_device)


scheduler.set_timesteps(num_inference_steps)

latents = latents * scheduler.init_noise_sigma

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for i, t in zip(range(0, 1001), tqdm(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)

    latent_model_input = scheduler.scale_model_input(latent_model_input, t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, t, latents).prev_sample

    if (i + 1) % 100 == 0:
        display_sample(latents, i + 1)

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
    image = vae.decode(latents).sample

In [ ]:
# convert the image to PIL
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
img = pil_images[0]
img.save(f"/data/junlei/diffusion-pipeline/generate-picture/1.png")
img